In [2]:
import os
import math

import pretty_midi
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder

from scipy.io import wavfile
from python_speech_features import mfcc
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, TensorDataset
#from torcheval.metrics import 

from accelerate import Accelerator

# Preproccessing

In [2]:
raw_data_folder = 'Data/'

In [3]:
def preprocess_midi_into_mel_and_mfcc(midi_file, segment_length=2000, num_cepstral=13):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    audio_data = midi_data.synthesize()
    wavfile.write("temp.wav", 44100, audio_data.astype(np.float32))

    audio, rate = librosa.load("temp.wav")
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=rate)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    mfcc_features = mfcc(audio, samplerate=rate, numcep=num_cepstral, winlen=0.025, winstep=0.01, nfft=1103)

    # Truncate or pad the spectrogram and MFCC features
    min_length = min(spectrogram_db.shape[1], mfcc_features.shape[0])
    if min_length % segment_length != 0:
        min_length = min_length - (min_length % segment_length)

    segments = []
    for start in range(0, min_length - segment_length + 1, segment_length):
        mel_segment = spectrogram_db[:, start:start + segment_length]
        mfcc_segment = mfcc_features[start:start + segment_length, :]
        segments.append((mel_segment, mfcc_segment))

    os.remove("temp.wav")

    return segments

In [4]:
def preprocess_data_in_directory(base_dir):
    X = []
    y = []
    X_mel = []
    X_mfcc = []

    # to show progression
    total_files = sum([len(files) for r, d, files in os.walk(base_dir)])
    processed_files = 0

    composers = os.listdir(base_dir)
    for composer in composers:
        composer_dir = os.path.join(base_dir, composer)
        if os.path.isdir(composer_dir):
            for file in os.listdir(composer_dir):
                if file.endswith('.mid'):
                    file_path = os.path.join(composer_dir, file)
                    try:
                        segments = preprocess_midi_into_mel_and_mfcc(file_path)

                        for mel_segment, mfcc_segment in segments:
                            X_mel.append(mel_segment)
                            X_mfcc.append(mfcc_segment)
                            y.append(composer)
                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")
                    processed_files += 1
                    if processed_files % (total_files // 10) == 0:
                        print(f"Processed {processed_files / total_files * 100}% of files")
    return np.array(X_mel), np.array(X_mfcc), np.array(y)

In [5]:
X_train_mel, X_train_mfcc, y_train = preprocess_data_in_directory(raw_data_folder + 'train')
X_test_mel, X_test_mfcc, y_test = preprocess_data_in_directory(raw_data_folder + 'test')
X_dev_mel, X_dev_mfcc, y_dev = preprocess_data_in_directory(raw_data_folder + 'dev')

Processed 9.788359788359788% of files
Processed 19.576719576719576% of files
Processed 29.365079365079367% of files


C:\Users\anton\anaconda3\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Processed 39.15343915343915% of files
Processed 48.94179894179894% of files
Processed 58.730158730158735% of files
Processed 68.51851851851852% of files
Processed 78.3068783068783% of files
Processed 88.09523809523809% of files
Processed 9.090909090909092% of files
Processed 18.181818181818183% of files
Processed 27.27272727272727% of files
Processed 36.36363636363637% of files
Processed 45.45454545454545% of files
Processed 54.54545454545454% of files
Processed 63.63636363636363% of files
Processed 72.72727272727273% of files
Processed 9.090909090909092% of files
Processed 18.181818181818183% of files
Processed 27.27272727272727% of files
Processed 36.36363636363637% of files
Processed 45.45454545454545% of files
Processed 54.54545454545454% of files
Processed 63.63636363636363% of files
Processed 72.72727272727273% of files


## Encoding Composer Labels

In [6]:
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_dev = le.transform(y_dev)


## Standardizing Input Labels

In [7]:
mean_mel = X_train_mel.mean()
std_mel = X_train_mel.std()

mean_mfcc = X_train_mfcc.mean()
std_mfcc = X_train_mfcc.std() 

In [8]:
X_train_mel = (X_train_mel - mean_mel) / std_mel
X_train_mfcc = (X_train_mfcc - mean_mfcc) / std_mfcc

X_dev_mel = (X_dev_mel - mean_mel) / std_mel
X_dev_mfcc = (X_dev_mfcc - mean_mfcc) / std_mfcc

X_test_mel = (X_test_mel - mean_mel) / std_mel
X_test_mfcc = (X_test_mfcc - mean_mfcc) / std_mfcc

## Saving Results

In [9]:
# Numpy to Tensor
X_train_mel = torch.from_numpy(X_train_mel).float()
X_train_mfcc = torch.from_numpy(X_train_mfcc).float()

X_train_mel = X_train_mel.unsqueeze(1)
X_train_mfcc = X_train_mfcc.unsqueeze(1)

y_train = torch.from_numpy(y_train).float()

X_test_mel = torch.from_numpy(X_test_mel).float()
X_test_mfcc = torch.from_numpy(X_test_mfcc).float()

X_test_mel = X_test_mel.unsqueeze(1)
X_test_mfcc = X_test_mfcc.unsqueeze(1)

y_test = torch.from_numpy(y_test).float()

X_dev_mel = torch.from_numpy(X_dev_mel).float()
X_dev_mfcc = torch.from_numpy(X_dev_mfcc).float()

X_dev_mel = X_dev_mel.unsqueeze(1)
X_dev_mfcc = X_dev_mfcc.unsqueeze(1)

y_dev = torch.from_numpy(y_dev).float()

In [10]:
# Saving Results
torch.save(X_train_mel, raw_data_folder + 'X_train_mel.pt')
torch.save(X_train_mfcc, raw_data_folder + 'X_train_mfcc.pt')
torch.save(y_train, raw_data_folder + 'y_train.pt')
torch.save(X_test_mel, raw_data_folder + 'X_test_mel.pt')
torch.save(X_test_mfcc, raw_data_folder + 'X_test_mfcc.pt')
torch.save(y_test, raw_data_folder + 'y_test.pt')
torch.save(X_dev_mel, raw_data_folder + 'X_dev_mel.pt')
torch.save(X_dev_mfcc, raw_data_folder + 'X_dev_mfcc.pt')
torch.save(y_dev, raw_data_folder + 'y_dev.pt')

# Feature Extraction

From the preproccessing section the MFCC and Mel data was extracted from audio files

## Loading Data

In [5]:
raw_data_folder = 'Data/'

In [6]:
X_train_mel = torch.load(raw_data_folder + 'X_train_mel.pt')
X_train_mfcc = torch.load(raw_data_folder + 'X_train_mfcc.pt')
y_train = torch.load(raw_data_folder + 'y_train.pt')

X_test_mel = torch.load(raw_data_folder + 'X_test_mel.pt')
X_test_mfcc = torch.load(raw_data_folder + 'X_test_mfcc.pt')
y_test = torch.load(raw_data_folder + 'y_test.pt')

X_dev_mel = torch.load(raw_data_folder + 'X_dev_mel.pt')
X_dev_mfcc = torch.load(raw_data_folder + 'X_dev_mfcc.pt')
y_dev = torch.load(raw_data_folder + 'y_dev.pt')

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
#X_train_mel = X_train_mel.to(device)
#X_train_mfcc = X_train_mfcc.to(device)
y_train = y_train.to(device, dtype=torch.int64)
#X_test_mel = X_test_mel.to(device)
#X_test_mfcc = X_test_mfcc.to(device)
y_test = y_test.to(device, dtype=torch.int64)
#X_dev_mel = X_dev_mel.to(device)
#X_dev_mfcc = X_dev_mfcc.to(device)
y_dev = y_dev.to(device, dtype=torch.int64)

## Getting the Class List

In [8]:
class_list = []
composers = os.listdir(raw_data_folder + 'train')
for composer in composers:
    class_list.append(composer)
class_list = class_list[:-1]

In [9]:
class_list

['bach',
 'bartok',
 'byrd',
 'chopin',
 'handel',
 'hummel',
 'mendelssohn',
 'mozart']

## Visualizing Data

In [ ]:
# Code to Visualize

## Weighted Random Sampling

In [10]:
# Get unique values and their counts
unique_values, counts = torch.unique(y_train.cpu(), return_counts=True)

# Print unique values and their counts
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

0: 234
1: 111
2: 194
3: 221
4: 197
5: 500
6: 227
7: 425
8: 265


In [11]:
# Setting Class Weights
target = torch.Tensor.numpy(y_train.cpu())
target = target.astype('int')
class_sample_counts = np.array([len(np.where(target == t)[0]) for t in np.unique(target)])
class_weights = [1/class_sample_counts[i] for i in target]
len(class_weights)

2374

In [12]:
sampler = WeightedRandomSampler(weights = class_weights, num_samples = len(class_weights), replacement = True)

## Making Mel DataLoader

In [84]:
class MelDataset(Dataset):
    def __init__(self, X_mel, y, class_list):
        self.X_mel = X_mel
        self.y = y
        self.class_list = class_list

    def __len__(self):
        return len(self.X_mel)

    def __getitem__(self, idx):
        return self.X_mel[idx], self.y[idx]


    def classes(self):
        # Return the list of class labels
        return self.class_list

In [85]:
train_dataset_mel = MelDataset(X_train_mel, y_train, class_list)
test_dataset_mel = MelDataset(X_test_mel, y_test, class_list)
val_dataset_mel = MelDataset(X_dev_mel, y_dev, class_list)

In [86]:
# Without Weighted Sampling

# Define the batch size
batch_size = 32

# Create the DataLoader
train_loader_mel = DataLoader(train_dataset_mel, batch_size=batch_size, shuffle=True)
test_loader_mel = DataLoader(test_dataset_mel, batch_size=batch_size, shuffle=False)
val_loader_mel = DataLoader(val_dataset_mel, batch_size=batch_size, shuffle=False)

In [87]:
# With Weighted Sampling
weighted_train_loader_mel = DataLoader(train_dataset_mel, sampler = sampler,  batch_size=batch_size)

## Making MFCC Dataloader

In [88]:
class MFCCDataset(Dataset):
    def __init__(self, X_mfcc, y, class_list):
        self.X_mfcc = X_mfcc
        self.y = y
        self.class_list = class_list

    def __len__(self):
        return len(self.X_mfcc)

    def __getitem__(self, idx):
        return self.X_mfcc[idx], self.y[idx]


    def classes(self):
        # Return the list of class labels
        return self.class_list

In [89]:
X_train_mfcc = X_train_mfcc.squeeze(1)
X_test_mfcc = X_test_mfcc.squeeze(1)
X_dev_mfcc = X_dev_mfcc.squeeze(1)

In [90]:
train_dataset_mfcc = MFCCDataset(X_train_mfcc, y_train, class_list) 
test_dataset_mfcc = MFCCDataset(X_test_mfcc, y_test, class_list) 
val_dataset_mfcc = MFCCDataset(X_dev_mfcc, y_dev, class_list) 

In [91]:
# Without Weighted Sampling
train_loader_mfcc = DataLoader(train_dataset_mfcc, batch_size=batch_size, shuffle=True)
test_loader_mfcc = DataLoader(test_dataset_mfcc, batch_size=batch_size, shuffle=False)
val_loader_mfcc = DataLoader(val_dataset_mfcc, batch_size=batch_size, shuffle=False)

In [92]:
# With Weighted Sampling
weighted_train_loader_mfcc = DataLoader(train_dataset_mfcc, sampler = sampler,  batch_size=batch_size)

## Making MEL and MFCC Dataloader

In [93]:
class CombinedDataset(Dataset):
    def __init__(self, X_mel, X_mfcc, y, class_list):
        self.X_mel = X_mel
        self.X_mfcc = X_mfcc
        self.y = y
        self.class_list = class_list

    def __len__(self):
        return len(self.X_mfcc)

    def __getitem__(self, idx):
        return self.X_mel[idx], self.X_mfcc[idx], self.y[idx]

    def classes(self):
        # Return the list of class labels
        return self.class_list

In [94]:
train_dataset_combined = CombinedDataset(X_train_mel, X_train_mfcc, y_train, class_list)
test_dataset_combined = CombinedDataset(X_test_mel, X_test_mfcc, y_test, class_list)
val_dataset_combined = CombinedDataset(X_dev_mel, X_dev_mfcc, y_dev, class_list)

In [95]:
# Without Weighted Sampling
train_loader_combined = DataLoader(train_dataset_combined, batch_size=batch_size, shuffle=True)
test_loader_combined = DataLoader(test_dataset_combined, batch_size=batch_size, shuffle=False)
val_loader_combined = DataLoader(val_dataset_combined, batch_size=batch_size, shuffle=False)

In [96]:
# With Weighted Sampling
weighted_train_loader_combined = DataLoader(train_dataset_combined, sampler = sampler,  batch_size=batch_size)

# Model Building

In [77]:
class MEL_CNN(nn.Module):
    def __init__(self, num_hidden_layers, init_filters, num_classes, dropout_rate=0.2):
        super(MEL_CNN, self).__init__()
        self.hidden_layers = num_hidden_layers
        
        #Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Input Layer
        self.layers = nn.ModuleDict()
        self.layers["input_Conv"] = nn.Conv2d(1, init_filters, kernel_size=3, padding=1)
        self.layers["input_batch_norm"] = nn.BatchNorm2d(init_filters)
        
        # Hidden Layers
        for i in range(num_hidden_layers):
            in_channels = 2 ** i * init_filters
            out_channels = in_channels * 2
            self.layers[f"hidden_{i}"] = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1)
            self.layers[f"batch_norm_{i}"] = nn.BatchNorm2d(out_channels)
        
        # Last Layer
        final_in_channels = 2 ** num_hidden_layers * init_filters
        final_out_channels = final_in_channels * 4
        self.layers["output_Conv"] = nn.Conv2d(final_in_channels, final_out_channels, kernel_size = 1)
        self.layers["output_batch_norm"] = nn.BatchNorm2d(final_out_channels)
        self.layers["linear"] = nn.Linear(final_out_channels, 128)
        self.layers["dropout"] = nn.Dropout(p = dropout_rate)
        self.layers["linear_output_1"] = nn.Linear(128, 64)
        self.layers["linear_output_2"] = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.layers["input_batch_norm"](self.layers["input_Conv"](x))))
        for i in range(self.hidden_layers):
            x = self.pool(nn.functional.relu(self.layers[f"batch_norm_{i}"](self.layers[f"hidden_{i}"](x))))
            
        x = self.pool(nn.functional.relu(self.layers["output_batch_norm"](self.layers["output_Conv"](x))))
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.layers["linear"](x))
        x = self.layers["dropout"](x)
        x = nn.functional.relu(self.layers["linear_output_1"](x))
        x = self.layers["linear_output_2"](x)
        return nn.functional.softmax(x, dim = 1)


In [37]:
class MFCC_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(MFCC_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)  # Multiply by 2 because it's bidirectional
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)  # Multiply by 2 because it's bidirectional

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # shape = (batch_size, seq_length, hidden_size*2)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [118]:
class CNN_LSTM_Ensemble(nn.Module):
    def __init__(self, model_CNN, model_LSTM, num_class):
        super(CNN_LSTM_Ensemble, self).__init__()
        self.LSTM = model_LSTM
        self.CNN = model_CNN
        
        self.outward = nn.Linear(num_class, num_class)
    
    def forward(self, MEL, MFCC):
        output_CNN = self.CNN(MEL)
        output_LSTM = self.LSTM(MFCC)
        output = output_CNN + output_LSTM
        
        output = self.outward(output)
        return nn.functional.softmax(output, dim = 1)

# Model Training

In [105]:
def train_single(model, optimizer, scheduler, train_loader, test_loader, num_epochs=100, patience=10):

    criterion = nn.CrossEntropyLoss()
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []
    best_test_loss = float('inf')
    epochs_no_improve = 0

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        train_iter = iter(train_loader)
        test_iter = iter(test_loader)

        for batch_idx, (mel_inputs, labels) in enumerate(train_iter):
            mel_inputs, labels = mel_inputs.to(device), labels.to(device)
            # Forward pass
            outputs = model(mel_inputs)

            # Compute loss
            labels = labels.long()
            loss = criterion(outputs, labels)

            # Compute accuracy
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            # Backward pass and optimization
            optimizer.zero_grad()
            accelerator.backward(loss)
            optimizer.step()
            train_loss += loss.item() * mel_inputs.size(0)

        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        train_accuracy = train_correct / train_total
        train_accuracies.append(train_accuracy)

        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)

        # Evaluation
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            test_correct = 0
            test_total = 0

            for mel_inputs, labels in test_loader:
                mel_inputs, labels = mel_inputs.to(device), labels.to(device)
                # Forward pass
                labels = labels.long()
                outputs = model(mel_inputs)
                _, predicted = torch.max(outputs.data, 1)

                # Compute accuracy and loss
                test_loss += criterion(outputs, labels).item() * mel_inputs.size(0)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

            test_loss /= len(test_loader.dataset)
            test_losses.append(test_loss)

            test_accuracy = test_correct / test_total
            test_accuracies.append(test_accuracy)

        accuracy = 100 * test_correct / test_total
        print(f'Epoch {epoch}, train loss: {train_loss:.4f}, train accuracy: {train_accuracy:.2f}, test loss: {test_loss:.4f}, test accuracy: {test_accuracy:.2f}, learning rate: {scheduler.get_lr()[0]}')

        if test_loss < best_test_loss:
            best_test_loss = test_loss
            epochs_no_improve = 0
            print('Saving model!')
            torch.save(model.state_dict(), 'cnn_lsmt_model.pt')
        else:
            epochs_no_improve += 1
            if epochs_no_improve in [20, 30, 40]:
                print('Stepping lr_scheduler')
                scheduler.step()
            if epochs_no_improve == patience:
                print('Early stopping!')
                break

    return train_losses, test_losses, train_accuracies, test_accuracies, best_test_loss

In [104]:
def train_ensemble(model, optimizer, scheduler, train_loader, test_loader, num_epochs=100, patience=10, model_id = "Best_Model.pt"):
    criterion = nn.CrossEntropyLoss()
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []
    best_test_loss = float('inf')
    epochs_no_improve = 0

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_idx, (mel_inputs, mfcc_inputs, labels) in enumerate(train_loader):
            mel_inputs, mfcc_inputs, labels = mel_inputs.to(device), mfcc_inputs.to(device), labels.to(device)
            # Forward pass
            outputs = model(mel_inputs, mfcc_inputs)

            # Compute loss
            loss = criterion(outputs, labels)

            # Compute accuracy
            _, predicted = torch.max(outputs.data, 1)
            labels = labels.long()
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            # Backward pass and optimization
            optimizer.zero_grad()
            accelerator.backward(loss)
            optimizer.step()
            train_loss += loss.item() * mel_inputs.size(0)

        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        train_accuracy = train_correct / train_total
        train_accuracies.append(train_accuracy)

        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)

        # Evaluation
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            test_correct = 0
            test_total = 0

            for mel_inputs, mfcc_inputs, labels in test_loader:
                mel_inputs, mfcc_inputs, labels = mel_inputs.to(device), mfcc_inputs.to(device), labels.to(device)
                # Forward pass
                outputs = model(mel_inputs, mfcc_inputs)
                _, predicted = torch.max(outputs.data, 1)

                # Compute accuracy and loss
                test_loss += criterion(outputs, labels).item() * mel_inputs.size(0)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

            test_loss /= len(test_loader.dataset)
            test_losses.append(test_loss)

            test_accuracy = test_correct / test_total
            test_accuracies.append(test_accuracy)

        accuracy = 100 * test_correct / test_total
        print(f'Epoch {epoch}, train loss: {train_loss:.4f}, train accuracy: {train_accuracy:.2f}, test loss: {test_loss:.4f}, test accuracy: {test_accuracy:.2f}, learning rate: {scheduler.get_lr()[0]}')

        # if epoch > 5 and train_loss > 2.1:
        #   print('Too little progress being made')
        #   break

        if test_loss < best_test_loss:
            best_test_loss = test_loss
            epochs_no_improve = 0
            print('Saving model!')
            torch.save(model.state_dict(), model_id)
        else:
            epochs_no_improve += 1
            if epochs_no_improve in [20, 30, 40]:
                print('Stepping lr_scheduler')
                scheduler.step()
            if epochs_no_improve == patience:
                print('Early stopping!')
                break

    return train_losses, test_losses, train_accuracies, test_accuracies, best_test_loss
    
    

## Test CNN Model

In [31]:
hl = 2
init = 8
num_classes = len(torch.unique(y_train.cpu()))
dropout = 0.5
test_model = MEL_CNN(hl, init, num_classes, dropout)

print(test_model)

CNN(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layers): ModuleDict(
    (input_Conv): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (input_batch_norm): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (hidden_0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm_0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (hidden_1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (output_Conv): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
    (output_batch_norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear): Linear(in_features=128, out_features=128, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (linear_output_1): Linear(in_features=128, out

In [35]:
patience = 10
num_epochs = 50
lr = 0.001
weight_decay=0.005
optimizer = torch.optim.Adam(test_model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

accelerator = Accelerator()
test_model, optimizer, weighted_train_loader_mel, test_loader_mel = accelerator.prepare(
test_model, optimizer, weighted_train_loader_mel, test_loader_mel
)

In [40]:
train_losses, test_losses, train_acc, test_acc, best_loss = train_single(test_model, optimizer, scheduler, weighted_train_loader_mel, test_loader_mel, num_epochs, patience)

Epoch 0, train loss: 1.9713, train accuracy: 0.40, test loss: 2.0165, test accuracy: 0.38, learning rate: 0.001
Saving model!
Epoch 1, train loss: 1.9622, train accuracy: 0.41, test loss: 2.0151, test accuracy: 0.34, learning rate: 0.001
Saving model!
Epoch 2, train loss: 2.0005, train accuracy: 0.36, test loss: 2.0101, test accuracy: 0.35, learning rate: 0.001
Saving model!
Epoch 3, train loss: 1.9742, train accuracy: 0.40, test loss: 1.9875, test accuracy: 0.37, learning rate: 0.001
Saving model!
Epoch 4, train loss: 1.9774, train accuracy: 0.40, test loss: 1.9772, test accuracy: 0.40, learning rate: 0.001
Saving model!
Epoch 5, train loss: 1.9762, train accuracy: 0.40, test loss: 1.9916, test accuracy: 0.38, learning rate: 0.001
Epoch 6, train loss: 1.9750, train accuracy: 0.40, test loss: 1.9766, test accuracy: 0.40, learning rate: 0.001
Saving model!
Epoch 7, train loss: 1.9761, train accuracy: 0.40, test loss: 1.9733, test accuracy: 0.41, learning rate: 0.001
Saving model!
Epoch 

## Test LSTM Model

In [100]:
input_size = 13
hidden_size = 32
num_layers = 2
num_classes = len(torch.unique(y_train.cpu()))

lstm_test_model = MFCC_LSTM(input_size, hidden_size, num_layers, num_classes)

print(lstm_test_model)

MFCC_LSTM(
  (lstm): LSTM(13, 32, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=64, out_features=9, bias=True)
)


In [101]:
patience = 10
num_epochs = 50
lr = 0.001
weight_decay = 0.01

optimizer = torch.optim.Adam(lstm_test_model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

accelerator = Accelerator()
lstm_test_model, optimizer, weighted_train_loader_mfcc, test_loader_mfcc = accelerator.prepare(
lstm_test_model, optimizer, weighted_train_loader_mfcc, test_loader_mfcc
)

In [64]:
train_losses, test_losses, train_acc, test_acc, best_loss = train_single(lstm_test_model, optimizer, scheduler, weighted_train_loader_mfcc, test_loader_mfcc, num_epochs, patience)

Epoch 0, train loss: 2.1945, train accuracy: 0.14, test loss: 2.1974, test accuracy: 0.11, learning rate: 0.001
Saving model!
Epoch 1, train loss: 2.1787, train accuracy: 0.17, test loss: 2.1882, test accuracy: 0.16, learning rate: 0.001
Saving model!
Epoch 2, train loss: 2.1482, train accuracy: 0.17, test loss: 2.1825, test accuracy: 0.13, learning rate: 0.001
Saving model!
Epoch 3, train loss: 2.1066, train accuracy: 0.18, test loss: 2.1300, test accuracy: 0.15, learning rate: 0.001
Saving model!
Epoch 4, train loss: 2.1332, train accuracy: 0.16, test loss: 2.1464, test accuracy: 0.10, learning rate: 0.001
Epoch 5, train loss: 2.0835, train accuracy: 0.19, test loss: 2.1280, test accuracy: 0.10, learning rate: 0.001
Saving model!
Epoch 6, train loss: 2.0502, train accuracy: 0.20, test loss: 2.1192, test accuracy: 0.13, learning rate: 0.001
Saving model!
Epoch 7, train loss: 2.1069, train accuracy: 0.17, test loss: 2.1305, test accuracy: 0.14, learning rate: 0.001
Epoch 8, train loss:

## Testing Ensemble Model 

In [111]:
# Mel Model
hl = 2
init = 8
num_classes = len(torch.unique(y_train.cpu()))
dropout = 0.5
mel_model = MEL_CNN(hl, init, num_classes, dropout)

print(mel_model)

mel_model = mel_model.to(device)

MEL_CNN(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layers): ModuleDict(
    (input_Conv): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (input_batch_norm): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (hidden_0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm_0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (hidden_1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (output_Conv): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
    (output_batch_norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear): Linear(in_features=128, out_features=128, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (linear_output_1): Linear(in_features=128,

In [112]:
# MFCC model
input_size = 13
hidden_size = 32
num_layers = 2
num_classes = len(torch.unique(y_train.cpu()))

mfcc_model = MFCC_LSTM(input_size, hidden_size, num_layers, num_classes)

print(mfcc_model)
mfcc_model = mfcc_model.to(device)

MFCC_LSTM(
  (lstm): LSTM(13, 32, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=64, out_features=9, bias=True)
)


In [120]:
# Ensemble
test_ensemble =  CNN_LSTM_Ensemble(mel_model, mfcc_model, num_classes)
print(test_ensemble)
test_ensemble = test_ensemble.to(device)

CNN_LSTM_Ensemble(
  (LSTM): MFCC_LSTM(
    (lstm): LSTM(13, 32, num_layers=2, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=64, out_features=9, bias=True)
  )
  (CNN): MEL_CNN(
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (layers): ModuleDict(
      (input_Conv): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (input_batch_norm): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (hidden_0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm_0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (hidden_1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (output_Conv): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      (output_batch_norm): BatchNorm2d(128, eps=1e-05, m

In [121]:
patience = 10
num_epochs = 50
lr = 0.001
weight_decay = 0.01

optimizer = torch.optim.Adam(test_ensemble.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

accelerator = Accelerator()
test_ensemble, optimizer, weighted_train_loader_combined, test_loader_combined = accelerator.prepare(
test_ensemble, optimizer, weighted_train_loader_combined, test_loader_combined
)

In [122]:
train_losses, test_losses, train_acc, test_acc, best_loss = train_ensemble(test_ensemble, optimizer, scheduler, weighted_train_loader_combined, test_loader_combined, num_epochs, patience)

Epoch 0, train loss: 2.1979, train accuracy: 0.10, test loss: 2.1982, test accuracy: 0.05, learning rate: 0.001
Saving model!


NameError: name 'model_id' is not defined

## Training CNN on MEL 

In [391]:
patience = 60
num_epochs = 200
lr = 0.0001
dropout_rate = 0.5
weight_decay=0.005
num_classes = len(torch.unique(y_train))
# Initialize the model
mel_model = Mel_CNN(num_classes, dropout_rate)
mel_model = mel_model.to(device)

In [392]:
optimizer = torch.optim.Adam(mel_model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [393]:
accelerator = Accelerator()
mel_model, optimizer, train_loader_mel, test_loader_mel = accelerator.prepare(
mel_model, optimizer, train_loader_mel, test_loader_mel
)

In [394]:
train_losses, test_losses = train_mel(mel_model, optimizer, scheduler, train_loader_mel, test_loader_mel, num_epochs, patience)

Epoch 0, train loss: 2.1830, train accuracy: 0.16, test loss: 2.1567, test accuracy: 0.20, learning rate: 0.0001
Saving model!
Epoch 1, train loss: 2.1369, train accuracy: 0.24, test loss: 2.0900, test accuracy: 0.36, learning rate: 0.0001
Saving model!
Epoch 2, train loss: 2.0782, train accuracy: 0.30, test loss: 2.0360, test accuracy: 0.39, learning rate: 0.0001
Saving model!
Epoch 3, train loss: 2.0460, train accuracy: 0.32, test loss: 2.0097, test accuracy: 0.38, learning rate: 0.0001
Saving model!
Epoch 4, train loss: 2.0115, train accuracy: 0.37, test loss: 2.0636, test accuracy: 0.28, learning rate: 0.0001
Epoch 5, train loss: 1.9847, train accuracy: 0.39, test loss: 2.0048, test accuracy: 0.37, learning rate: 0.0001
Saving model!
Epoch 6, train loss: 1.9604, train accuracy: 0.42, test loss: 1.9548, test accuracy: 0.43, learning rate: 0.0001
Saving model!
Epoch 7, train loss: 1.9310, train accuracy: 0.47, test loss: 1.9188, test accuracy: 0.49, learning rate: 0.0001
Saving model

## Training CNN_LSTM Ensemble

In [11]:
patience = 60
num_epochs = 20
num_classes = len(torch.unique(y_train))

lrs = [0.01, 0.001, 0.0001, 0.00001]
dropout_rates = [0.1, 0.15, 0.2, 0.3, 0.4, 0.5]
weight_decays = [0.001, 0.005, 0.01, 0.05]
hidden_sizes = [32, 64, 128, 256, 512, 1024]
num_lstm_layer_options = [1, 2, 3]
num_cnn_layer_options = [3, 4, 5]
initial_filter_options = [8, 16, 32, 64]

In [23]:
best_loss = float('inf')
best_hyperparameters = None
hyperparameters_tested = set()

In [26]:
def run_training(train_loader_mel, test_loader_mel, num_classes, device, num_epochs, patience, accelerator):
    global hyperparameters_tested
    global best_loss
    global best_hyperparameters

    def generate_hypers():
        lr = np.random.choice(lrs)
        dropout_rate = np.random.choice(dropout_rates)
        weight_decay = np.random.choice(weight_decays)
        hidden_size = np.random.choice(hidden_sizes)
        num_lstm_layers = np.random.choice(num_lstm_layer_options)
        num_cnn_layers = np.random.choice(num_cnn_layer_options)
        initial_filter = np.random.choice(initial_filter_options)

        hyperparameters_dict = {'lr': lr, 'dropout_rate': dropout_rate, 'weight_decay': weight_decay,
                          'hidden_size': hidden_size, 'num_lstm_layers': num_lstm_layers,
                          'num_cnn_layers': num_cnn_layers, 'initial_filter': initial_filter}

    return hyperparameters_dict


    hyperparameters_dict = generate_hypers()
    while tuple(hyperparameters_dict.values()) in hyperparameters_tested:
        hyperparameters_dict = generate_hypers()

    hyperparameters_tested.add(tuple(hyperparameters_dict.values()))

  print(hyperparameters_dict)

  mel_lstm_cnn_model = Mel_LSTM_CNN(num_classes,
                                    hyperparameters_dict['hidden_size'],
                                    hyperparameters_dict['initial_filter'],
                                    hyperparameters_dict['num_lstm_layers'],
                                    hyperparameters_dict['num_cnn_layers'],
                                    hyperparameters_dict['dropout_rate'])
  mel_lstm_cnn_model = mel_lstm_cnn_model.to(device)
  optimizer = torch.optim.Adam(mel_lstm_cnn_model.parameters(), lr=hyperparameters_dict['lr'], weight_decay=hyperparameters_dict['weight_decay'])
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

  mel_lstm_cnn_model, optimizer, train_loader_mel, test_loader_mel = accelerator.prepare(
    mel_lstm_cnn_model, optimizer, train_loader_mel, test_loader_mel
  )


  _, _, _, _, best_test_loss = train_mel(mel_lstm_cnn_model,
    optimizer, scheduler, train_loader_mel, test_loader_mel, num_epochs, patience)



  if best_test_loss < best_loss:
    best_loss = best_test_loss
    best_hyperparameters = hyperparameters_dict
    print(f'Best hyperparameters updated! Best loss: {best_loss}')

  return hyperparameters_dict, best_loss, best_hyperparameters

In [ ]:
num_trials = 60
for i in range(num_trials):
  accelerator = Accelerator()
  hyperparameters, best_loss, best_hyperparameters = run_training(train_loader_mel, test_loader_mel, num_classes, device, num_epochs, patience, accelerator)
  print(f'Run {i+1}/{num_trials} completed. Best loss so far: {best_loss}, Best hyperparameters so far: {best_hyperparameters}')

{'lr': 0.0001, 'dropout_rate': 0.1, 'weight_decay': 0.01, 'hidden_size': 128, 'num_lstm_layers': 3, 'num_cnn_layers': 5, 'initial_filter': 8}
Epoch 0, train loss: 2.1971, train accuracy: 0.12, test loss: 2.1951, test accuracy: 0.28, learning rate: 0.0001
Saving model!
Epoch 1, train loss: 2.1965, train accuracy: 0.21, test loss: 2.1945, test accuracy: 0.28, learning rate: 0.0001
Saving model!
Epoch 2, train loss: 2.1961, train accuracy: 0.21, test loss: 2.1940, test accuracy: 0.28, learning rate: 0.0001
Saving model!
Epoch 3, train loss: 2.1956, train accuracy: 0.21, test loss: 2.1935, test accuracy: 0.28, learning rate: 0.0001
Saving model!
Epoch 4, train loss: 2.1952, train accuracy: 0.21, test loss: 2.1931, test accuracy: 0.28, learning rate: 0.0001
Saving model!
Epoch 5, train loss: 2.1949, train accuracy: 0.21, test loss: 2.1927, test accuracy: 0.28, learning rate: 0.0001
Saving model!
Epoch 6, train loss: 2.1945, train accuracy: 0.21, test loss: 2.1922, test accuracy: 0.28, learn

Best loss so far: 1.938269287971273, Best hyperparameters so far: {'lr': 0.0001, 'dropout_rate': 0.4, 'weight_decay': 0.005, 'hidden_size': 64, 'num_lstm_layers': 1, 'num_cnn_layers': 5, 'initial_filter': 16}

# Evaluation

In [ ]:
def calculate_accuracy(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for mel_inputs, labels in test_loader:
            mel_inputs = mel_inputs.to(device)  # Move inputs to the device
            labels = labels.to(device)  # Move labels to the device

            # Forward pass
            outputs = model(mel_inputs)
            _, predicted = torch.max(outputs.data, 1)

            # Update counts
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

# Usage example
val_accuracy = calculate_accuracy(model, val_loader)
print(f'Validation Accuracy: {val_accuracy:.4f}')


In [ ]:
def create_confusion_matrix(model, test_loader, num_classes):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for mel_inputs, labels in test_loader:
            mel_inputs = mel_inputs.to(device)  # Move inputs to the device
            labels = labels.to(device)  # Move labels to the device

            # Forward pass
            outputs = model(mel_inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Create the confusion matrix
    cm = confusion_matrix(all_labels, all_predictions, labels=np.arange(num_classes))
    return cm

def plot_confusion_matrix(confusion_mtx, labels):
    # Normalize the confusion matrix
    confusion_mtx = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))

    # Create a heatmap using seaborn
    sns.heatmap(confusion_mtx, annot=True, fmt='.2f', cmap='Blues', xticklabels=labels, yticklabels=labels, ax=ax)

    # Set labels and title
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')

    # Rotate x-axis labels if needed
    plt.xticks(rotation=45)

    # Show the plot
    plt.show()

In [ ]:
num_classes = len(torch.unique(y_dev))
confusion_mtx = create_confusion_matrix(model, val_loader, num_classes)
class_labels = val_loader.dataset.classes()
plot_confusion_matrix(confusion_mtx, class_labels)

In [ ]:
mel_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader_mel:
        inputs = inputs.to(device)  # Move inputs to the device (e.g., GPU)
        labels = labels.to(device)  # Move labels to the device

        # Forward pass
        outputs = mel_model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        # Update counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total

print(f'Validation Accuracy: {accuracy:.4f}')

In [ ]:
mel_model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader_mel:
        inputs = inputs.to(device)  # Move inputs to the device (e.g., GPU)
        labels = labels.to(device)  # Move labels to the device

        # Forward pass
        outputs = mel_model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Create the confusion matrix
cm = confusion_matrix(all_labels, all_predictions, labels=np.arange(num_classes))

# Normalize the confusion matrix
confusion_mtx = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 8))


class_labels = val_loader.dataset.classes()
# Create a heatmap using seaborn
sns.heatmap(confusion_mtx, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels, ax=ax)

# Set labels and title
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Show the plot
plt.show()